# Basketball challenge

In [1]:
from PIL import Image
import torch
import torchvision
from Dataset import Basketball
from Dataprocess import Preprocess
from modules import LinearNet
import random
from torch.utils.data import DataLoader
%reload_ext autoreload

# Image transformation

In [2]:
img_transformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64,48)),
    torchvision.transforms.CenterCrop((48,48)),
    torchvision.transforms.ToTensor()
])


# Preprocessing Dataset

In [3]:
dataprocess = Preprocess().background_subtractor

# Loading Dataset

In [4]:
path = "data"
dataset = Basketball(path, split='training', num_frame = 100, img_transform = img_transformation, dataprocess=dataprocess)
trainset, testset = dataset.train_test_split()

In [5]:
trainset = DataLoader(trainset, shuffle=True)
testset = DataLoader(testset, shuffle=True)

# Display Video

In [ ]:
from IPython import display
import torchvision.transforms.functional as F
view1 = dataset.__getitem__(0)[1]
for index in range(1):
    view1 = dataset.__getitem__(index)[0]
    for idx, img in enumerate(view1):
        img1 = F.to_pil_image(img)
        display.display(img1.resize((320,240)), Image.NEAREST)
        display.clear_output(wait=True)

# CUDA

In [6]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True

# Linear Model

In [13]:
print(dataset[0][0][0].shape)
variables = dataset[0][0][0].numel()
net = LinearNet(variables, 2) # only 2 classifier hit or miss as output and variables input as parameters
variables

torch.Size([100, 3, 48, 48])


691200

In [14]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.001,momentum=0.4, nesterov=True)
loss_fn = torch.nn.CrossEntropyLoss().to(device)
net = net.to(device)

In [32]:
def train(net, trainset):
    net.train()
    for batch, target in trainset:
        #batch is taken only as view1
        #change is later for both views
        inputs = batch[0][0].view(1,-1).to(device)
        target = torch.as_tensor(target).to(device)
        output = net(inputs)
        optimizer.zero_grad()
        l = loss_fn(output, target)
        l.backward()
        #print(l)   

In [33]:
 def test(net, testset):
    net.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for batch, target in testset:
            inputs = batch[0][0].view(1,-1).to(device)
            target = torch.as_tensor(target).to(device)
            output = net(inputs)

            eval_loss += loss_fn(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    eval_loss /= 35
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        eval_loss, correct, len(testset),
        100. * correct / len(testset)))

In [34]:
epochs = 10
for epoch in range(1, epochs+1):
    train(net, trainset)
    test(net, testset)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)


Test set: Average loss: 0.5533, Accuracy: 16/28 (57%)

